<a href="https://colab.research.google.com/github/kristinazk/HomeworkWeek13/blob/main/animal_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# dataset https://www.kaggle.com/datasets/alessiocorrado99/animals10

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import shutil
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
Name = 'animal_prediction'
tensorboard = TensorBoard(log_dir = 'logs' )

Copying Our dataset from Google Drive to Colab Working Dir, for faster train

In [ ]:
shutil.copytree("/content/drive/MyDrive/raw-img", "/content/data")

'/content/data'

In [ ]:
seed_train_validation = 1 # Must be same for train_ds and val_ds
shuffle_value = True
validation_split = 0.3
BATCH_SIZE = 128
IMG_SIZE = (224, 224)

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
directory ='./data',
image_size = IMG_SIZE,
crop_to_aspect_ratio=True,
validation_split = validation_split,
subset = "training",
seed = seed_train_validation,
color_mode = 'rgb',
shuffle = shuffle_value,
batch_size = BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
directory ='./data',
image_size = IMG_SIZE,
crop_to_aspect_ratio=True,
validation_split = validation_split,
subset = "validation",
seed = seed_train_validation,
color_mode = 'rgb',
shuffle = shuffle_value,
batch_size = BATCH_SIZE)


Found 26179 files belonging to 10 classes.
Using 18326 files for training.
Found 26179 files belonging to 10 classes.
Using 7853 files for validation.


In [ ]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_dataset = val_ds.take(val_batches // 5)
validation_dataset = val_ds.skip(val_batches // 5)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_ds.prefetch(buffer_size=AUTOTUNE)
validation_dataset = val_ds.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomBrightness(factor=0.2)
])

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False


In [ ]:
base_model.summary()


Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()


In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256,activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(10)(x)
model = tf.keras.Model(inputs, output)


In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.evaluate(validation_dataset)

62/62 [==============================] - 29s 261ms/step - loss: 2.5262 - accuracy: 0.0754


[2.5261919498443604, 0.0753852054476738]

In [ ]:
model.evaluate(test_dataset)

12/12 [==============================] - 6s 369ms/step - loss: 2.5040 - accuracy: 0.0781


[2.504045248031616, 0.078125]

In [ ]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                             

In [ ]:
history = model.fit(train_ds,
                    epochs=10,
                    validation_data=validation_dataset, callbacks = [tensorboard])


Epoch 1/10
144/144 [==============================] - 70s 431ms/step - loss: 1.8224 - accuracy: 0.4266 - val_loss: 0.6262 - val_accuracy: 0.9107
Epoch 2/10
144/144 [==============================] - 61s 408ms/step - loss: 0.9102 - accuracy: 0.7169 - val_loss: 0.3248 - val_accuracy: 0.9387
Epoch 3/10
144/144 [==============================] - 64s 437ms/step - loss: 0.6814 - accuracy: 0.7905 - val_loss: 0.2432 - val_accuracy: 0.9465
Epoch 4/10
144/144 [==============================] - 62s 414ms/step - loss: 0.5884 - accuracy: 0.8157 - val_loss: 0.2096 - val_accuracy: 0.9510
Epoch 5/10
144/144 [==============================] - 57s 385ms/step - loss: 0.5440 - accuracy: 0.8305 - val_loss: 0.1942 - val_accuracy: 0.9535
Epoch 6/10
144/144 [==============================] - 57s 390ms/step - loss: 0.4957 - accuracy: 0.8470 - val_loss: 0.1806 - val_accuracy: 0.9557
Epoch 7/10
144/144 [==============================] - 61s 408ms/step - loss: 0.4702 - accuracy: 0.8518 - val_loss: 0.1695 - val_ac

In [ ]:
model.evaluate(test_dataset)

12/12 [==============================] - 6s 367ms/step - loss: 0.1600 - accuracy: 0.9577


[0.1600073128938675, 0.9576823115348816]